In [2]:
import pandas as pd
import requests
import io
import random
import os
from utils import download_image, download_data_export

In [11]:
IMAGES_DIR = "./images"
BEFORE_PADDING_DIR = f"{IMAGES_DIR}/before_padding_cropped"
AFTER_PADDING_DIR = f"{IMAGES_DIR}/after_padding_cropped"
FULL_IMAGES_DIR = f"{IMAGES_DIR}/full_images"

# Create directories if they don't exist
for directory in [IMAGES_DIR, BEFORE_PADDING_DIR, AFTER_PADDING_DIR, FULL_IMAGES_DIR]:
    os.makedirs(directory, exist_ok=True)

In [4]:
success, df = download_data_export()
if not success:
    raise Exception("No data downloaded, check your credentials in .env file")

Saved 10522 rows to cleaned_export.csv


In [ ]:
from threading import Thread, Condition, Event
from queue import Queue

def download_image(url):
    response = requests.get(url)
    return response.content

def save_image(content, file_name):
    with open(file_name, 'wb') as f:
        f.write(content)

def pad_image(img_path, color=(255, 255, 255)):
    im = Image.open(img_path)
    width, height = im.size
    if width == height: 
        return im
    max_dim = max(width, height)
    pad_img = Image.new(im.mode, (max_dim, max_dim), color)
    pad_img.paste(im, ((max_dim - width) // 2, (max_dim - height) // 2))
    return pad_img

In [15]:

def pad_worker(pad_queue, download_path, download_done_event):
    while not download_done_event.is_set() or not pad_queue.empty():
        if not pad_queue.empty():
            count, idx, row = pad_queue.get()
            name = row['specimenId_unique']
            ind_image_path = os.path.join(download_path, f"{name}.jpg")
            save_image_path = os.path.join(AFTER_PADDING_DIR, f"{name}.jpg")
            try:
                padded_image = pad_image(ind_image_path)
                padded_image.save(save_image_path)
                print(f"Padded image {count}")
            except Exception as e:
                print(f"Error padding image {count}: {e}")

download_done_event = Event()
pad_queue = Queue()
pad_thread = Thread(target=pad_worker, args=(pad_queue, BEFORE_PADDING_DIR, download_done_event))
pad_thread.start()

for idx, row in df.iterrows():
    image_url = row['image']
    specimen_id = row['specimenId_unique']
    file_name = f"{BEFORE_PADDING_DIR}/{specimen_id}.jpg"
    if os.path.exists(file_name):
        continue
    content = download_image(image_url)
    save_image(content, file_name)
    pad_queue.put((idx, idx, row))
    

download_done_event.set()
pad_thread.join()




2707 Jovanprokopack&&10-19-2022&&G017_03 http://137.184.182.201:5005/images/session/image-2022-10-19-15-54-08.743-459d004632e3d0c1-1-19.jpg
Error padding image 2707: name 'Image' is not defined


FileNotFoundError: [Errno 2] No such file or directory: './images/before_padding_cropped/08/16/2022&&08-16-2022&&l036_02.jpg'

In [14]:


def download_worker(download_queue, pad_queue, curr_df, download_path, download_done_event):
    while not download_queue.empty():
        count, idx, row = download_queue.get()
        name = row['specimenId_unique']
        url = row['image']
        ind_image_path = os.path.join(download_path, f"{name}.jpg")

        if os.path.exists(ind_image_path):
            print(count, "exists")
            continue

        try:
            content = download_image(url)
            save_image(content, ind_image_path)
            print(count, name, url)
            pad_queue.put((count, idx, row))
        except Exception as e:
            print(f"Error downloading image {count}: {e}")
    download_done_event.set()



curr_df = pd.read_csv('cleaned_export.csv')
# select only the last 10 rows
curr_df = curr_df.iloc[-10:]

download_queue = Queue()
pad_queue = Queue()
download_done_event = Event()

for idx, row in curr_df.iterrows():
    download_queue.put((idx, idx, row))

download_thread = Thread(target=download_worker, args=(download_queue, pad_queue, curr_df, BEFORE_PADDING_DIR, download_done_event))
pad_thread = Thread(target=pad_worker, args=(pad_queue, BEFORE_PADDING_DIR, download_done_event))

download_thread.start()
pad_thread.start()

download_thread.join()
pad_thread.join()


64 technicalvalidation&&03-23-2022&&BC-34 http://137.184.182.201:5005/images/session/image-2022-03-23-13-29-04.795-2ac016f01d4d6c16-2-5.jpg
Error padding image 64: name 'Image' is not defined
0 exists
1 exists
2 exists
3 exists
4 exists
5 exists
6 exists
7 exists
8 exists
9 exists
10 exists
11 exists
12 exists
13 exists
14 exists
15 exists
16 exists
17 exists
18 exists
19 exists
20 exists
21 exists
22 exists
23 exists
24 exists
25 exists
26 exists
27 exists
28 exists
29 exists
30 exists
31 exists
32 exists
33 exists
34 exists
35 exists
36 exists
37 exists
38 exists
39 exists
40 exists
41 exists
42 exists
43 exists
44 exists
45 exists
46 exists
47 exists
48 exists
49 exists
50 exists
51 exists
52 exists
53 exists
54 exists
55 exists
56 exists
57 exists
58 exists
59 exists
60 exists
61 exists
62 exists
63 exists
64 exists
65 technicalvalidation&&03-23-2022&&Bc-59 http://137.184.182.201:5005/images/session/image-2022-03-23-13-28-38.108-2ac016f01d4d6c16-3-36.jpg
Error padding image 65: nam

KeyboardInterrupt: 

 MukonoPSC&&09-10-2022&&MKNG013_03 http://137.184.182.201:5005/images/session/image-2022-09-22-07-24-23.172-f67276ffbafcaa8b-20-377.jpg
2231Error padding image 2230: name 'Image' is not defined MukonoPSC&&09-10-2022&&MKNG013 http://137.184.182.201:5005/images/session/image-2022-09-22-07-24-23.018-f67276ffbafcaa8b-20-375.jpg

2231Error padding image 2231: name 'Image' is not defined exists

